Import from our main project and also import the `reload` method if we decide to hot-reload a module

In [2]:
import sys
sys.path.extend(['../server', '../client'])

import benchmark 
format_f1_scores = benchmark.format_f1_scores
f1_scores = benchmark.f1_scores

from importlib import reload


Perform training on GPU

In [3]:
import torch 
device = torch.device("cuda")

Import data (do not use NumPy arrays this time)

In [4]:
import csv
from collections import defaultdict

from sklearn import metrics
from sklearn.model_selection import train_test_split

SEED = 1337

with open('../data/atis/train.tsv', 'rt') as f:
    reader = csv.reader(f, delimiter='\t')
    data = list(reader)
    data_x = [row[0] for row in data]
    data_y = [row[1] for row in data]

data_x_train, data_x_val, data_y_train, data_y_val = train_test_split(data_x, data_y, test_size=0.2, random_state=SEED)

def val_accuracy(f):
    print(metrics.accuracy_score(data_y_val, f(data_x_val)))

data_x_words = sorted({w for x in data_x_train for w in x.split(' ')})

by_label = defaultdict(list)
for x, y in zip(data_x_train, data_y_train):
    by_label[y].append(x)
labels = set(by_label)

SCORE_DATA = { "train": [data_x_train, data_y_train], "val": [data_x_val, data_y_val]}

def confusion(model, on_data : tuple[list[str], list[str]]):
    cm = defaultdict(lambda: defaultdict(int))
    for x, y in zip(*on_data):
        cm[y][model(x)] += 1
    return cm

def print_f1_scores(model, data_key="train"):
    print(format_f1_scores(f1_scores(confusion(model, SCORE_DATA[data_key]))))


Import the same transformers classifiers as the last time and use the GPU:

In [17]:
from transformers import pipeline

tr_model_names = [
    "MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli", # 100m parameters
    "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", # 200m parameters
    "facebook/bart-large-mnli", # 400m parameters
]

tr_models = [pipeline("zero-shot-classification", model=name, device=device) 
             for name in tr_model_names]

BATCH_SIZE = 128

def into_my_model(transformer_model):
    def as_model(xs):
        results = transformer_model(xs, batch_size=BATCH_SIZE, candidate_labels=sorted(labels))
        return [result['labels'][0] for result in results]
    return as_model

my_models = [into_my_model(model) for model in tr_models] 

Perform a quick test:

In [18]:
SCORE_DATA["2"] = ['fly to nyc', 'take a taxi to penn station'], ['flight', 'ground_service']
my_models[0](SCORE_DATA["2"][0])

['flight', 'capacity']

Let's use a more professional-looking context manager when measuring time:



In [23]:
import time

class Timer:
    def __enter__(self):
        self.start = time.time()
        self.laps = []
        return self

    def __exit__(self, *args):
        pass
    
    @property 
    def elapsed(self):
        return time.time() - self.start
    
    def lap(self):
        self.laps.append(self.elapsed)
        self.start += self.laps[-1]    

    @property 
    def average_lap(self):
        return sum(self.laps) / len(self.laps)

# Usage example:
with Timer() as t:
    assert t.elapsed < 0.1
    time.sleep(0.5)
    assert 0.4 < t.elapsed < 0.6
    t.lap()
    assert t.elapsed < 0.1
    assert len(t.laps) == 1
    t.lap()
    assert 0.2 < t.average_lap < 0.3


Convert our naive arrays to GPU tensors

In [12]:
from datasets import Dataset

def convert_to_gpu_dataset(x):
    d = Dataset.from_dict({'data': x})
    return d.with_format("torch", device=device)

SCORE_DATASET = {
    key: [convert_to_gpu_dataset(d) for d in values] 
    for key, values in SCORE_DATA.items()
}    

In [20]:
def confusion_batch(model, on_data : tuple[Dataset, Dataset]):
    results = model(on_data[0]['data'])
    cm = defaultdict(lambda: defaultdict(int))
    for y, y_ in zip(on_data[1]['data'], results):
        cm[y][y_] += 1
    return cm
        

Now that we use the GPU for inference and the datasets are preloaded,
we can check the inference time and the validation results.

In [24]:
with Timer() as t:
    for model in my_models:
        print(format_f1_scores(f1_scores(confusion_batch(model, SCORE_DATASET["train"]))))
        t.lap()
  
    print("this took {} seconds".format(t.laps))


  abbreviation (56, 115, 37): 0.42
  aircraft (30, 7, 28): 0.63
  airfare (14, 3, 305): 0.083
  airfare+flight_time (0, 0, 1): 0.0
  airline (70, 59, 48): 0.57
  airline+flight_no (0, 1, 1): 0.0
  airport (12, 65, 1): 0.27
  capacity (6, 66, 5): 0.14
  cheapest (1, 149, 0): 0.013
  city (5, 3, 10): 0.43
  distance (7, 153, 6): 0.081
  flight (2259, 135, 496): 0.88
  flight+airfare (9, 122, 5): 0.12
  flight_no (0, 31, 9): 0.0
  flight_time (17, 15, 27): 0.45
  ground_fare (1, 118, 13): 0.015
  ground_service (8, 0, 177): 0.083
  meal (2, 45, 0): 0.082
  quantity (0, 46, 37): 0.0
  restriction (4, 73, 0): 0.099
  AVERAGE (20): 0.22
  abbreviation (86, 87, 7): 0.65
  aircraft (10, 914, 48): 0.02
  airfare (229, 1340, 90): 0.24
  airfare+flight_time (0, 7, 1): 0.0
  airline (0, 65, 118): 0.0
  airline+flight_no (0, 16, 1): 0.0
  airport (7, 25, 6): 0.31
  capacity (2, 2, 9): 0.27
  cheapest (0, 3, 1): 0.0
  city (1, 2, 14): 0.11
  distance (9, 549, 4): 0.032
  flight (81, 17, 2674): 0.057

In [22]:
with Timer() as t:
    for model in my_models:
        print(format_f1_scores(f1_scores(confusion_batch(model, SCORE_DATASET["val"]))))
        t.lap()
  
    print("this took {} seconds".format(t.laps))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  abbreviation (8, 24, 7): 0.34
  aircraft (12, 2, 8): 0.71
  aircraft+flight+flight_no (0, 0, 1): 0.0
  airfare (6, 0, 78): 0.13
  airline (18, 15, 12): 0.57
  airline+flight_no (0, 0, 1): 0.0
  airport (5, 21, 0): 0.32
  capacity (3, 15, 2): 0.26
  cheapest (0, 45, 0): 0.0
  city (2, 3, 1): 0.5
  distance (1, 50, 6): 0.034
  flight (548, 38, 123): 0.87
  flight+airfare (2, 31, 1): 0.11
  flight_no (0, 8, 3): 0.0
  flight_time (2, 1, 6): 0.36
  ground_fare (0, 36, 3): 0.0
  ground_service (1, 0, 49): 0.039
  ground_service+ground_fare (0, 0, 1): 0.0
  meal (4, 7, 0): 0.53
  quantity (0, 9, 12): 0.0
  restriction (1, 9, 0): 0.18
  AVERAGE (21): 0.24
  abbreviation (12, 20, 3): 0.51
  aircraft (4, 231, 16): 0.031
  aircraft+flight+flight_no (0, 0, 1): 0.0
  airfare (63, 336, 21): 0.26
  airfare+flight_time (0, 2, 0): 0.0
  airline (0, 12, 30): 0.0
  airline+flight_no (0, 4, 1): 0.0
  airport (2, 9, 3): 0.25
  capacity (0, 0, 5): 0.0
  city (0, 2, 3): 0.0
  distance (6, 135, 1): 0.081
  

Let's also try this 33m parameter model:

In [25]:
small_model_name = "pszemraj/xtremedistil-l12-h384-uncased-zeroshot-v1.1"
small_model = into_my_model(pipeline("zero-shot-classification", device=device, model = small_model_name))

config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [28]:
def time_my_models(model_list, key:str="val"):
    with Timer() as t:
        for model in model_list:
            print(format_f1_scores(f1_scores(confusion_batch(model, SCORE_DATASET[key]))))
            t.lap()
      
        print("this took {} seconds".format(t.laps))

time_my_models([small_model, my_models[0]])

  abbreviation (11, 4, 4): 0.73
  aircraft (17, 46, 3): 0.41
  aircraft+flight+flight_no (0, 0, 1): 0.0
  airfare (22, 176, 62): 0.16
  airfare+flight_time (0, 2, 0): 0.0
  airline (18, 159, 12): 0.17
  airline+flight_no (0, 6, 1): 0.0
  airport (5, 8, 0): 0.56
  capacity (0, 0, 5): 0.0
  cheapest (0, 1, 0): 0.0
  city (0, 18, 3): 0.0
  distance (4, 23, 3): 0.24
  flight (326, 14, 345): 0.64
  flight+airfare (0, 12, 3): 0.0
  flight_no (0, 0, 3): 0.0
  flight_time (1, 0, 7): 0.22
  ground_fare (0, 0, 3): 0.0
  ground_service (40, 2, 10): 0.87
  ground_service+ground_fare (0, 0, 1): 0.0
  meal (2, 0, 2): 0.67
  quantity (1, 2, 11): 0.13
  restriction (1, 6, 0): 0.25
  AVERAGE (22): 0.23
  abbreviation (8, 24, 7): 0.34
  aircraft (12, 2, 8): 0.71
  aircraft+flight+flight_no (0, 0, 1): 0.0
  airfare (6, 0, 78): 0.13
  airline (18, 15, 12): 0.57
  airline+flight_no (0, 0, 1): 0.0
  airport (5, 21, 0): 0.32
  capacity (3, 15, 2): 0.26
  cheapest (0, 45, 0): 0.0
  city (2, 3, 1): 0.5
  dista

In [30]:
tr_model_names = [small_model_name, *tr_model_names]
my_models = [small_model, *tr_model_names]

Let's examine the models more closely:

In [40]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
auto_models = [AutoModelForSequenceClassification.from_pretrained(n) for n in tr_model_names]
auto_tokenizers = [AutoTokenizer.from_pretrained(n) for n in tr_model_names]

In [41]:
for m in auto_models:
    print(m)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12,

In [43]:
very_small_model_name = "MoritzLaurer/xtremedistil-l6-h256-zeroshot-v1.1-all-33"
very_small_model = into_my_model(pipeline("zero-shot-classification", device=device, model = small_model_name))

time_my_models([very_small_model])

  abbreviation (11, 4, 4): 0.73
  aircraft (17, 46, 3): 0.41
  aircraft+flight+flight_no (0, 0, 1): 0.0
  airfare (22, 176, 62): 0.16
  airfare+flight_time (0, 2, 0): 0.0
  airline (18, 159, 12): 0.17
  airline+flight_no (0, 6, 1): 0.0
  airport (5, 8, 0): 0.56
  capacity (0, 0, 5): 0.0
  cheapest (0, 1, 0): 0.0
  city (0, 18, 3): 0.0
  distance (4, 23, 3): 0.24
  flight (326, 14, 345): 0.64
  flight+airfare (0, 12, 3): 0.0
  flight_no (0, 0, 3): 0.0
  flight_time (1, 0, 7): 0.22
  ground_fare (0, 0, 3): 0.0
  ground_service (40, 2, 10): 0.87
  ground_service+ground_fare (0, 0, 1): 0.0
  meal (2, 0, 2): 0.67
  quantity (1, 2, 11): 0.13
  restriction (1, 6, 0): 0.25
  AVERAGE (22): 0.23
this took [4.776393413543701] seconds


In [45]:
tr_model_names

['pszemraj/xtremedistil-l12-h384-uncased-zeroshot-v1.1',
 'MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli',
 'MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli',
 'facebook/bart-large-mnli']